In [1]:
from classes.reddit_scrapper import RedditScrapper
import pandas as pd

In [2]:
rs = RedditScrapper()
rs.set_current_subreddit('dankmemes')

In [3]:
score_df = rs.get_latest_scores()

100%|██████████| 13/13 [00:07<00:00,  1.72it/s]

13 data points have been gathered in runtime so far.



In [4]:
score_df.head(20)

,final_score,raw_score,author,num_memes_in_bottom,num_memes_in_top,spammer_index,highest_upvotes,highest_upvotes_score,lowest_ratio,timstamp
0,91.457023,7.232888,JKdoe,0,4,0.000000,64747,1.944325,0.93,1573406682
1,88.618051,6.316350,2Pac_if,2,10,0.200000,125679,2.398614,0.79,1573406682
2,80.071772,4.200692,orqh,1,5,0.200000,56860,1.866974,0.90,1573406682
3,78.680209,3.930763,kayneshaw,3,9,0.333333,109809,2.298692,0.76,1573406682
4,77.751187,3.760281,RtfMc,4,11,0.363636,60065,1.899132,0.90,1573406682
5,74.771503,3.261806,poppy38,0,2,0.000000,44735,1.735003,0.94,1573406682
6,73.210593,3.027638,Drarckfort,0,2,0.000000,35995,1.627762,0.93,1573406682
7,73.072066,3.007686,ARAKSH,2,6,0.333333,30244,1.550354,0.97,1573406682
8,73.026168,3.001105,DearLeader42,0,2,0.000000,30002,1.546961,0.97,1573406682
9,71.902230,2.844359,staticdanger,1,3,0.333333,65872,1.954886,0.97,1573406682
